# 5-1. 결정 트리 (p.220)
> 결정 트리 알고리즘
> 
선형이 아닌 데이터는? 
레드와인인지 화이트화인인지 라벨링을 해보자. 

In [1]:
# 먼저 로지스틱 회귀로 와인을 분류해보자

In [2]:
import pandas as pd

red_wine = pd.read_csv('data/winequality-red.csv', sep=';')
# wine의 데이터가 구분점이 ,(콤마)가 아닌 :(세미콜론)으로 나눠져있어서 => 코드에 구분점이 (;) 라고 알려줌.

white_wine = pd.read_csv('data/winequality-white.csv', sep=';')

In [3]:
red_wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
white_wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [5]:
# 필요한 열만 가져오기 
red_wine = red_wine[['alcohol', 'residual sugar', 'pH']]

# 컬럼 이름 변경
red_wine.rename(columns={'residual sugar': 'sugar'}, inplace=True)

# 새로운 컬럼(class) 추가
red_wine['class'] = 0
# 레드 와인이라면 class가 0
# 데이터 확인
red_wine.head()


,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0
1,9.8,2.6,3.20,0
2,9.8,2.3,3.26,0
3,9.8,1.9,3.16,0
4,9.4,1.9,3.51,0


In [6]:
white_wine = white_wine[['alcohol', 'residual sugar', 'pH']]
white_wine.rename(columns={'residual sugar': 'sugar'}, inplace=True)
white_wine['class'] = 1

# 화이트 와인이라면 class가 1

In [7]:
# 이렇게 만든 red_wine 데이터와 white_wine 데이터를 하나로 합치기 
# concat

wine = pd.concat([red_wine, white_wine])
wine.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6497 entries, 0 to 4897
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   int64  
dtypes: float64(3), int64(1)
memory usage: 253.8 KB


In [8]:
# 데이터를 랜덤하게 분류해보자 
# 학습과 정답 데이터로 

data = wine[['alcohol', 'sugar', 'pH']]
target = wine[['class']]

In [9]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = \
train_test_split(data, target)

In [10]:
# 로지스틱 회귀를 먼저 해보기 위해
# 표준 점수화 부터 해주자

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [11]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_scaled, train_target)
print(lr.score(train_scaled, train_target))

print(lr.score(test_scaled, test_target))

0.7803776683087028
0.7784615384615384


C:\Users\박수똥\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# 무조건 도수가 낮다고 화이트 와인, 레드와인? 이라고 할 수 없다.
# 선형 관계를 띄지 않는다.
# => 그래서 이건 로지스틱 회귀로 표현하기에 맞지 않는다.
#
# => 결정 트리 모델을 사용하겠다. 